In [17]:
import pickle
import lightgbm as lgbm
import pandas as pd

run_type = "predict"

exclude = [
    'id',
 'year',
 'week',
 'season_type',
 'start_date',
 'completed',
 'home_team',
 'home_points',
 'home_post_wp',
 'away_team',
 'away_points',
 'away_post_wp',
 'home_ml',
 'away_ml',
 'spread_open',
 'over_under_open',
 'team_home',
 'team_away']

In [18]:
df = pd.read_parquet(path="./X_predict.parquet")

In [19]:
with open("model-2023-09-15.pickle", "rb") as f:
    model = pickle.load(f)

In [20]:
y_preds = model.predict(df.drop(columns=exclude))
y_preds = pd.Series(y_preds, name="preds")

In [21]:
if run_type == "predict":
    preds_df = df[["id", "home_team", "away_team"]].copy()
    preds_df["predicted"] = y_preds

In [22]:
preds_df.columns = ["id", "home", "away", "predicted"]

In [23]:
preds_df

,id,home,away,predicted
0,401525434,Notre Dame,Navy,-17.752208
1,401520145,Jacksonville State,UTEP,-10.155512
2,401532392,San Diego State,Ohio,-1.812098
3,401525450,New Mexico State,UMass,-13.273348
4,401520147,Vanderbilt,Hawai'i,-19.077994
...,...,...,...,...
784,401520444,UTEP,Liberty,7.988380
785,401520437,New Mexico State,Jacksonville State,6.593597
786,401532630,Nevada,Wyoming,6.331094
787,401532626,Hawai'i,Colorado State,2.465674


In [24]:
preds_df.to_clipboard(sep=",", index=False)